In [17]:
!git clone https://github.com/muxspace/facial_expressions.git

fatal: destination path 'facial_expressions' already exists and is not an empty directory.


In [ ]:
import csv
data = {}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader = csv.reader(f)
  next(reader)
  for row in reader:
    key = row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key] = [row[1]]

In [ ]:
data

{'anger': ['facial-expressions_2868588k.jpg',
  'Alejandro_Toledo_0004.jpg',
  'Alexander_Lukashenko_0001.jpg',
  'Alicia_Hollowell_0001.jpg',
  'Alvaro_Noboa_0002.jpg',
  'Al_Gore_0002.jpg',
  'Anastasia_Myskina_0001.jpg',
  'Anastasia_Myskina_0003.jpg',
  'Angelina_Jolie_0005.jpg',
  'Arnold_Schwarzenegger_0023.jpg',
  'Atal_Bihari_Vajpayee_0010.jpg',
  'Ben_Braun_0001.jpg',
  'Billy_Crystal_0004.jpg',
  'Billy_Donovan_0001.jpg',
  'Bill_Graham_0005.jpg',
  'Bob_Huggins_0001.jpg',
  'Bob_Huggins_0004.jpg',
  'Brian_Scalabrine_0001.jpg',
  'Bud_Selig_0004.jpg',
  'Carlos_Arroyo_0001.jpg',
  'Carlos_Ortega_0001.jpg',
  'Celine_Dion_0008.jpg',
  'Charles_Moose_0001.jpg',
  'Charles_Moose_0002.jpg',
  'Charles_Moose_0004.jpg',
  'Charles_Moose_0006.jpg',
  'Charles_Moose_0013.jpg',
  'Colin_Powell_0007.jpg',
  'Colin_Powell_0030.jpg',
  'Colin_Powell_0043.jpg',
  'Colin_Powell_0055.jpg',
  'Colin_Powell_0149.jpg',
  'Colin_Powell_0180.jpg',
  'Colin_Powell_0181.jpg',
  'Colin_Powell_0188

In [ ]:
emotion_list = list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [ ]:
import os
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [ ]:
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training/', emotion))
  os.mkdir(os.path.join('master_data/testing/', emotion))

In [ ]:
from shutil import copyfile
split_size = 0.8

for emotion, images in data.items():
  train_size = int(split_size*len(images))
  train_images = images[:train_size]
  test_images = images[train_size:]
  for image in train_images:
    source = os.path.join('/content/facial_expressions/images', image)
    dest = os.path.join('/content/master_data/training', emotion, image)
    copyfile(source, dest)
  for image in test_images:
    source = os.path.join('/content/facial_expressions/images', image)
    dest = os.path.join('/content/master_data/testing', emotion, image)
    copyfile(source, dest)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [ ]:
model = tf.keras.models.Sequential([
      Conv2D(16, (3,3), activation='relu', input_shape = (100, 100, 3)),
      MaxPooling2D(2, 2),
      Conv2D(32, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Conv2D(64, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Flatten(),
      Dense(1024, activation='relu'),
      Dense(8, activation='softmax')
])
model.compile(optimizer = Adam(lr = 0.01), loss = 'categorical_crossentropy', metrics = ['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 23, 23, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 10, 10, 64)        0

In [20]:
train_dir='/content/master_data/training'
test_dir='/content/master_data/testing'
train_datagen=ImageDataGenerator(rescale = 1.0/255)

## we initialise our object of image class and tell it to point to particular data source
##The ImageDataGenerator class in Keras provides a way to load images from a directory, apply various transformations to the images
## (such as rescaling, rotation, shearing, zooming, etc.), and yield batches of images along with their labels during the training process.
## Using an image data generator helps in efficiently loading and processing images in batches, which is beneficial for training deep learning models, especially when working with large datasets that cannot fit into memory.
train_generator = train_datagen.flow_from_directory(
                                                    train_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                  )

test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory(
                                                    test_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                  )

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [22]:
es = EarlyStopping(monitor='val_acc', patience = 2, min_delta=0.01)
##min_delta=0.01 specifies the minimum change in the monitored metric to qualify as an improvement. If the change in validation accuracy is less than 0.01, it will not be considered an improvement.

In [24]:
model.fit_generator(train_generator,
                    epochs=10,
                    verbose=1,
                    validation_data=test_generator,
                    callbacks=[es])

<ipython-input-24-860791d9b115>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/10
86/86 [==============================] - 154s 2s/step - loss: 0.9111 - acc: 0.6403 - val_loss: 1.8729 - val_acc: 0.6156
Epoch 2/10
86/86 [==============================] - 149s 2s/step - loss: 0.5762 - acc: 0.8094 - val_loss: 1.4664 - val_acc: 0.6597
Epoch 3/10
86/86 [==============================] - 149s 2s/step - loss: 0.5046 - acc: 0.8249 - val_loss: 1.5180 - val_acc: 0.6659
Epoch 4/10
86/86 [==============================] - 136s 2s/step - loss: 0.4584 - acc: 0.8392 - val_loss: 1.5190 - val_acc: 0.6849
Epoch 5/10
86/86 [==============================] - 144s 2s/step - loss: 0.4004 - acc: 0.8605 - val_loss: 1.7613 - val_acc: 0.6725
Epoch 6/10
86/86 [==============================] - 150s 2s/step - loss: 0.3674 - acc: 0.8685 - val_loss: 1.7227 - val_acc: 0.6802
